In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
data = pd.read_csv('InventoryPosition.csv',delimiter=';')
data.dtypes
#data['price'] = pd.to_numeric(data['price'], errors='coerce').fillna(0)

Date              object
StoreCode         object
ProductCode       object
SalesQuantity      int64
StoreStock         int64
IncomingStock      int64
SalesRevenue     float64
dtype: object

In [56]:
#data_array = np.array(data)
data['ProductPrice'] = data['SalesRevenue']/data['SalesQuantity']
d = data.groupby(["ProductCode", "ProductPrice"]).size()
d


ProductCode  ProductPrice
Product1     1.570393e+06     7
             7.970336e+08     2
             1.058871e+09     1
             1.112691e+09     7
             4.892682e+09     1
             4.892682e+09    23
             7.970336e+09    38
Product10    2.302966e+06    32
             1.170739e+09     3
             1.170739e+09    41
             7.163072e+09     2
             7.625205e+09     1
Product100   5.754136e+08     1
             6.115783e+08     5
             8.409486e+08     2
             4.456668e+09     1
             5.205939e+09     7
             5.347660e+09     1
             6.321176e+09     1
             6.349251e+09     1
             6.727437e+09     8
             6.895911e+09     1
             8.409482e+09    28
Product101   7.700330e+05    47
             5.005523e+07     1
             1.948801e+08     1
             5.137017e+08     1
             5.269090e+08    13
             5.269094e+08     1
             5.463546e+08     1
              